## 生成器
https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/0014317799226173f45ce40636141b6abc8424e12b5fb27000

通过列表生成式，我们可以直接创建一个列表。但是，受到内存限制，列表容量肯定是有限的。而且，创建一个包含100万个元素的列表，不仅占用很大的存储空间，如果我们仅仅需要访问前面几个元素，那后面绝大多数元素占用的空间都白白浪费了。

所以，如果列表元素可以按照某种算法推算出来，那我们是否可以在循环的过程中不断推算出后续的元素呢？这样就不必创建完整的list，从而节省大量的空间。在Python中，这种一边循环一边计算的机制，称为生成器：generator。

### 创建 generator 的两种方式
#### 第一种：把一个列表生成式的[]改成()

In [9]:
L = [x * x for x in range(5)]
g = (x * x for x in range(5))
print(L)
print(g)

[0, 1, 4, 9, 16]
<generator object <genexpr> at 0x00A09ED0>


创建L和g的区别仅在于最外层的[]和()，L是一个list，而g是一个generator。

我们可以直接打印出list的每一个元素，但我们怎么打印出generator的每一个元素呢？

如果要一个一个打印出来，可以通过next()函数获得generator的下一个返回值：

##### 通过next()函数获得generator的下一个返回值

In [10]:
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))

0
1
4
9
16


StopIteration: 

**generator保存的是算法**，每次调用next(g)，就计算出g的下一个元素的值，直到计算到最后一个元素，没有更多的元素时，抛出StopIteration的错误。

当然，上面这种不断调用next(g)实在是太变态了，正确的方法是使用for循环，因为generator也是可迭代对象：

In [11]:
g = (x * x for x in range(5))
for i in g:
    print(i)

0
1
4
9
16


所以，我们创建了一个generator后，基本上永远不会调用next()，而是通过for循环来迭代它，并且不需要关心StopIteration的错误。


#### 第二种：用 yield 
如果一个函数定义中包含yield关键字，那么这个函数就不再是一个普通函数，而是一个generator

以斐波拉契数为例，可以输出斐波那契数列的前N个数

In [12]:
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        print(b)
        a, b = b, a + b
        n = n + 1
    return 'done'

fib(4)

1
1
2
3


'done'

仔细观察，可以看出，fib函数实际上是定义了斐波拉契数列的推算规则，可以从第一个元素开始，推算出后续任意的元素，这种逻辑其实非常类似generator。

也就是说，上面的函数和generator仅一步之遥。要把fib函数变成generator，只需要把print(b)改为yield b就可以了：

In [18]:
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1
    return 'done'

f = fib(4)
print(f)
for i in f:
    print(i)

<generator object fib at 0x00A049C0>
1
1
2
3


这里，最难理解的就是generator和函数的执行流程不一样。函数是顺序执行，遇到return语句或者最后一行函数语句就返回。
**而变成generator的函数，在每次调用 next()的时候执行，遇到yield语句返回，再次执行时从上次返回的yield语句处继续执行。**

In [19]:
def odd():
    print('step 1')
    yield 1
    print('step 2')
    yield(3)
    print('step 3')
    yield(5)
    
o = odd()
print(next(o))
print(next(o))
print(next(o))
print(next(o))

step 1
1
step 2
3
step 3
5


StopIteration: 

odd不是普通函数，而是generator，在执行过程中，遇到yield就中断，下次又继续执行。执行3次yield后，已经没有yield可以执行了，所以，第4次调用next(o)就报错。

In [22]:
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1
    return 'done'

g = fib(4)
for n in fib(4):
    print(n)

1
1
2
3


**但是用for循环调用generator时，发现拿不到generator的return语句的返回值。**

如果想要拿到返回值，必须捕获StopIteration错误，返回值包含在StopIteration的value中：

In [29]:
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1
    return 'done'

g = fib(4)

print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))

1
1
2
3


StopIteration: done

In [31]:
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1
    return 'done'

g = fib(4)
while True:
    try:
        x = next(g)
        print(x)
    except StopIteration as e:
        print('Generator return value:',e.value)
        break

1
1
2
3
Generator return value: done


### yield 与 return
http://python.jobbole.com/81911/

#### 没有 return
在一个生成器中，如果没有return，则默认执行到函数完毕时返回StopIteration；

In [2]:
def g1():
    yield 1

g = g1()
print(next(g))
print(next(g))

1


StopIteration: 

- 第一次调用next(g)时，会在执行完yield语句后挂起，所以此时程序并没有执行结束。
- 程序试图从yield语句的下一条语句开始执行，发现已经到了结尾，所以抛出StopIteration异常。

#### 有return
如果遇到return,如果在执行过程中 return，则直接抛出 StopIteration 终止迭代。

In [3]:
def g2():
    yield 'a'
    return
    yield 'b'

g = g2()
print(next(g))
print(next(g))

a


StopIteration: 

- 程序停留在执行完yield 'a'语句后的位置。
- 程序发现下一条语句是return，所以抛出StopIteration异常，这样yield 'b'语句永远也不会执行。

#### 有带返回值的 return

如果在return后返回一个值，那么这个值为StopIteration异常的说明，不是程序的返回值。

生成器没有办法使用return来返回值。

**如果想要拿到返回值，必须捕获StopIteration错误，返回值包含在StopIteration的value中**

In [28]:
def g3():
    yield 'hello'
    return 'world'

g = g3()
print(next(g))
print(next(g))

hello


StopIteration: world

In [31]:
def g3():
    yield 'hello'
    return 'world'

try:
    g = g3()
    print(next(g))
    print(next(g))
except StopIteration as e:
    print('Generator return value:',e.value)


hello
Generator return value: world


### 生成器支持的方法

- close() 手动关闭生成器函数，后面的调用会直接返回StopIteration异常。
- send() 生成器函数最大的特点是可以接受外部传入的一个变量，并根据变量内容计算结果后返回。这是生成器函数最难理解的地方，也是最重要的地方，实现后面我会讲到的协程就全靠它了。

In [8]:
help(g)

Help on generator object:

g3 = class generator(object)
 |  Methods defined here:
 |  
 |  __del__(...)
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  close(...)
 |      close() -> raise GeneratorExit inside generator.
 |  
 |  send(...)
 |      send(arg) -> send 'arg' into generator,
 |      return next yielded value or raise StopIteration.
 |  
 |  throw(...)
 |      throw(typ[,val[,tb]]) -> raise exception in generator,
 |      return next yielded value or raise StopIteration.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  gi_code
 |  
 |  gi_frame
 |  
 |  gi_running
 |  
 |  gi_yieldfrom
 |      object being iterated by yield from, or None



#### close() 
手动关闭生成器函数，后面的调用会直接返回StopIteration异常。

In [11]:
def g4():
    yield 1
    yield 2
    yield 3

g = g4()
print(next(g))
g.close()
print(next(g))

1


StopIteration: 

#### send()

生成器函数最大的特点是可以接受外部传入的一个变量，并根据变量内容计算结果后返回。
这是生成器函数最难理解的地方，也是最重要的地方，实现后面我会讲到的协程就全靠它了。

In [12]:
def gen():
    value = 0
    while True:
        receive = yield value
        if receive == 'e':
            break
        value = 'got:{}'.format(receive)
        
g = gen()
print(g.send(None))
print(g.send('aaa'))
print(g.send(3))
print(g.send('e'))

0
got:aaa
got:3


StopIteration: 

其实receive=yield value包含了3个步骤： 
1、向函数外抛出（返回）value 
2、暂停(pause)，等待next()或send()恢复 
3、赋值receive=MockGetValue() 。 这个MockGetValue()是假想函数，用来接收send()发送进来的值


执行流程：

- **通过g.send(None)或者next(g)可以启动生成器函数**，并执行到第一个yield语句结束的位置。此时，执行完了yield语句，但是没有给receive赋值。yield value会输出初始值0
**注意：在启动生成器函数时只能send(None),如果试图输入其它的值都会得到错误提示信息。**
- 通过g.send(‘aaa’)，会传入aaa，并赋值给receive，然后计算出value的值，并回到while头部，执行yield value语句有停止。此时yield value会输出”got: aaa”，然后挂起。
- 通过g.send(3)，会重复第2步，最后输出结果为”got: 3″
当我们g.send(‘e’)时，程序会执行break然后推出循环，最后整个函数执行完毕，所以会得到StopIteration异常。

从上面可以看出， 在第一次send(None)启动生成器（执行1–>2，通常第一次返回的值没有什么用）之后，对于外部的每一次send()，生成器的实际在循环中的运行顺序是3–>1–>2，也就是先获取值，然后dosomething，然后返回一个值，再暂停等待。


#### throw()

用来向生成器函数送入一个异常，可以结束系统定义的异常，或者自定义的异常。
throw()后直接跑出异常并结束程序，或者消耗掉一个yield，或者在没有下一个yield的时候直接进行到程序的结尾。

In [1]:
def gen():
    while True:
        try:
            yield 'normal value'
            yield 'normal value 2'
            print('here')
        except ValueError:
            print('we got ValueError here')
        except TypeError:
            break
            
g = gen()
print(next(g))
print(g.throw(ValueError))
print(next(g))
print(g.throw(TypeError))

normal value
we got ValueError here
normal value
normal value 2


StopIteration: 

解释：

- print(next(g))：会输出normal value，并停留在yield ‘normal value 2’之前。
- 由于执行了g.throw(ValueError)，所以会跳过所有后续的try语句，也就是说yield ‘normal value 2’不会被执行，然后进入到except语句，打印出we got ValueError here。然后再次进入到while语句部分，消耗一个yield，所以会输出normal value。
- print(next(g))，会执行yield ‘normal value 2’语句，并停留在执行完该语句后的位置。
- g.throw(TypeError)：会跳出try语句，从而print(‘here’)不会被执行，然后执行break语句，跳出while循环，然后到达程序结尾，所以跑出StopIteration异常。

下面给出一个综合例子，用来把一个多维列表展开，或者说扁平化多维列表)

In [9]:
def flatten(nested):
    try:
        if isinstance(nested,str):
            raise TypeError
        for sublist in nested:
            for element in flatten(sublist):
                print('got:',element)
    except TypeError:
        yield nested
        
L=['aaadf',[1,2,3],2,4,[5,[6,[8,[9]],'ddf'],7]]

for num in flatten(L):
    print(num)

got: aaadf
got: 1
got: 2
got: 3
got: 2
got: 4
got: 5
got: 6
got: 8
got: 9
got: ddf
got: 7


### 总结

- 按照鸭子模型理论，生成器就是一种迭代器，可以使用for进行迭代。
- 第一次执行next(generator)时，会执行完yield语句后程序进行挂起，所有的参数和状态会进行保存。再一次执行next(generator)时，会从挂起的状态开始往后执行。在遇到程序的结尾或者遇到StopIteration时，循环结束。
- 可以通过generator.send(arg)来传入参数，这是协程模型。
- 可以通过generator.throw(exception)来传入一个异常。throw语句会消耗掉一个yield。可以通过generator.close()来手动关闭生成器。
- next()等价于send(None)

## Python协程：从yield/send到async/await
http://python.jobbole.com/86069/

### 从yield说起



Python并发编程之深入理解yield from语法（八）
https://www.cnblogs.com/wongbingming/p/9085268.html

简单应用：拼接可迭代对象

我们可以用一个使用yield和一个使用yield from的例子来对比看下。

使用yield

In [15]:
astr = 'ABC'
alist = [1,2,3]
adict={'name':'Jamie','age':18}
agen = (i for i in range(4,8))

def gen(*args,**kw):
    print(args)
    print(kw)
    for item in args:
        for i in item:
            yield i
            
newlist = gen(astr,alist,adict,agen)
print(list(newlist))

('ABC', [1, 2, 3], {'name': 'Jamie', 'age': 18}, <generator object <genexpr> at 0x00FFEDE0>)
{}
['A', 'B', 'C', 1, 2, 3, 'name', 'age', 4, 5, 6, 7]


使用yield from

In [16]:
astr = 'ABC'
alist = [1,2,3]
adict={'name':'Jamie','age':18}
agen = (i for i in range(4,8))

def gen(*args,**kw):
    print(args)
    print(kw)
    for item in args:
        yield from item
            
newlist = gen(astr,alist,adict,agen)
print(list(newlist))

('ABC', [1, 2, 3], {'name': 'Jamie', 'age': 18}, <generator object <genexpr> at 0x01157F00>)
{}
['A', 'B', 'C', 1, 2, 3, 'name', 'age', 4, 5, 6, 7]


可以看出，yield from后面加上可迭代对象，他可以把可迭代对象里的每个元素一个一个的yield出来，对比yield来说代码更加简洁，结构更加清晰。

复杂应用：生成器的嵌套

如果你认为只是 yield from 仅仅只有上述的功能的话，那你就太小瞧了它，它的更强大的功能还在后面。

当 yield from 后面加上一个生成器后，就实现了生成的嵌套。

当然实现生成器的嵌套，并不是一定必须要使用yield from，而是使用yield from可以让我们避免让我们自己处理各种料想不到的异常，而让我们专注于业务代码的实现。

如果自己用yield去实现，那只会加大代码的编写难度，降低开发效率，降低代码的可读性。既然Python已经想得这么周到，我们当然要好好利用起来。

讲解它之前，首先要知道这个几个概念

- 1、调用方：调用委派生成器的客户端（调用方）代码
- 2、委托生成器：包含yield from表达式的生成器函数
- 3、子生成器：yield from后面加的生成器函数

你可能不知道他们都是什么意思，没关系，来看下这个例子。

这个例子，是实现实时计算平均值的。

比如，第一次传入10，那返回平均数自然是10.
第二次传入20，那返回平均数是(10+20)/2=15
第三次传入30，那返回平均数(10+20+30)/3=20

In [18]:
# 子生成器
def average_gen():
    total = 0
    count = 0
    average = 0
    while True:
        new_num = yield average
        count += 1
        total += new_num
        average = total/count
        
# 委托生成器
def proxy_gen():
    while True:
        yield from average_gen()
        
# 调用方
def main():
    calc_average = proxy_gen()
    next(calc_average)
    print(calc_average.send(10))
    print(calc_average.send(20))
    print(calc_average.send(30))
    
if __name__ == '__main__':
    main()

10.0
15.0
20.0


In [19]:
# 子生成器
def average_gen():
    total = 0
    count = 0
    average = 0
    while True:
        new_num = yield average
        if new_num is None:
            break
        count += 1
        total += new_num
        average = total/count

    # 每一次return，都意味着当前协程结束。
    return total,count,average

# 委托生成器
def proxy_gen():
    while True:
        # 只有子生成器要结束（return）了，yield from左边的变量才会被赋值，后面的代码才会执行。
        total, count, average = yield from average_gen()
        print("计算完毕！！\n总共传入 {} 个数值， 总和：{}，平均数：{}".format(count, total, average))

# 调用方
def main():
    calc_average = proxy_gen()
    next(calc_average)            # 预激协程
    print(calc_average.send(10))  # 打印：10.0
    print(calc_average.send(20))  # 打印：15.0
    print(calc_average.send(30))  # 打印：20.0
    calc_average.send(None)      # 结束协程
    # 如果此处再调用calc_average.send(10)，由于上一协程已经结束，将重开一协程

if __name__ == '__main__':
    main()

10.0
15.0
20.0
计算完毕！！
总共传入 3 个数值， 总和：60，平均数：20.0


In [32]:
# 子生成器
def average_gen():
    total = 0
    count = 0
    average = 0
    while True:
        new_num = yield average
        if new_num is None:
            break
        count += 1
        total += new_num
        average = total/count

    # 每一次return，都意味着当前协程结束。
    return total,count,average

# 委托生成器
def proxy_gen():
    while True:
        # 只有子生成器要结束（return）了，yield from左边的变量才会被赋值，后面的代码才会执行。
        total, count, average = yield from average_gen()
        print("计算完毕！！\n总共传入 {} 个数值， 总和：{}，平均数：{}".format(count, total, average))

# 调用方
def main():
    calc_average = average_gen()
    next(calc_average)            # 预激协程
    print(calc_average.send(10))  # 打印：10.0
    print(calc_average.send(20))  # 打印：15.0
    print(calc_average.send(30))  # 打印：20.0
    calc_average.send(None)      # 结束协程


if __name__ == '__main__':
    main()

10.0
15.0
20.0


StopIteration: (60, 3, 20.0)

## python协程1：yield 10分钟入门

https://segmentfault.com/a/1190000009769387



### 协程生成器的基本行为
首先说明一下，协程有四个状态，可以使用inspect.getgeneratorstate(...)函数确定：

- GEN_CREATED # 等待开始执行
- GEN_RUNNING # 解释器正在执行（只有在多线程应用中才能看到这个状态）
- GEN_SUSPENDED # 在yield表达式处暂停
- GEN_CLOSED # 执行结束

In [40]:
import inspect

def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:',x)
    
my_coro = simple_coroutine()
print(inspect.getgeneratorstate(my_coro))

my_coro.send(None)
print(inspect.getgeneratorstate(my_coro))
my_coro.send(42)

GEN_CREATED
-> coroutine started
GEN_SUSPENDED
-> coroutine received: 42


StopIteration: 

- yield 在表达式中使用；如果协程只需要从客户那里接收数据，yield关键字右边不需要加表达式（yield默认返回None）
- send方法的参数会成为暂停yield表达式的值，所以，仅当协程处于暂停状态是才能调用send方法。如果协程还未激活（GEN_CREATED 状态）要调用next(my_coro) 激活协程，也可以调用my_coro.send(None)
- 最先调用my_coro.send(None) / next(my_coro) 这一步通常称为”预激“（prime）协程---即，让协程向前执行到第一个yield表达式，准备好作为活跃的协程使用。

### 再看一个两个值的协程

In [41]:
def coro(a):
    print('-> coroutine started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

mycoro = coro(14)
print(inspect.getgeneratorstate(mycoro))
mycoro.send(None)
print(inspect.getgeneratorstate(mycoro))
mycoro.send(28)
print(inspect.getgeneratorstate(mycoro))
mycoro.send(99)

GEN_CREATED
-> coroutine started: a = 14
GEN_SUSPENDED
-> Received: b = 28
GEN_SUSPENDED
-> Received: c = 99


StopIteration: 

## python协程2：yield from 从入门到精通

https://segmentfault.com/a/1190000009781688
